In [ ]:
project_path = "/media/vijinim/data/Experiments/Data/FUDAN/Uniprot-12/"

# kraken2 --confidence 0.1 
# https://www.biostars.org/p/402619/

In [ ]:
import re
import csv
import subprocess
import os
import gzip
import shutil
import glob
import xlsxwriter
import collections

from datetime import datetime, timedelta
from Bio import SeqIO
from collections import defaultdict

# Get contig lengths

In [ ]:
contig_lengths = {}

for index, record in enumerate(SeqIO.parse(project_path+"final.contigs.fa", "fasta")):
    contig_lengths[record.id] = len(record.seq)

# Get kraken result

In [ ]:
taxid_total_len = {}

species_names_taxid = {}

taxid_list = []

contig_taxid = {}

taxid_contigs = {}

taxid_to_species = {}

species_names_taxid_length = {}

with open(project_path+"MEGAHIT_new_kraken_res_0.1.txt", "r") as myfile:
    for line in myfile.readlines():
        strings = line.strip().split("\t")
        
        if strings[0] == "C":
            
            species_strings = strings[2].split(" ")
            
            if len(species_strings) > 3 and "unclassified" not in strings[2] and "complex" not in strings[2] and " group " not in strings[2] and "Human" not in strings[2] and "phage" not in strings[2] and "cellular organisms" not in strings[2]:
                
                taxid = species_strings[-1][:-1]
                
                my_species = ""
                
                if species_strings[0] == "Candidatus":
                    my_species = species_strings[0] + " " + species_strings[1] + " " + species_strings[2]
                    
                elif species_strings[1] != "sp.":
                    my_species = species_strings[0] + " " + species_strings[1]
                elif "(taxid" not in species_strings[3]:
                    my_species = strings[2].split("(taxid")[0]
                    
                my_species = my_species.replace("[", "")
                my_species = my_species.replace("]", "")
                
                if my_species != "":
                    
                    taxid_to_species[taxid] = my_species
                    contig_taxid[strings[1]] = taxid
                    
                    if taxid not in taxid_list:
                        taxid_list.append(taxid)
                        
                    if my_species in species_names_taxid:
                        species_names_taxid[my_species].add(taxid)
                    else:
                        species_names_taxid[my_species]=set([taxid])

                    if taxid not in taxid_contigs:
                        taxid_contigs[taxid] = [strings[1]]
                    else:
                        taxid_contigs[taxid].append(strings[1])
            
                    if taxid not in taxid_total_len:
                        taxid_total_len[taxid] = contig_lengths[strings[1]]
                    else:
                        taxid_total_len[taxid] += contig_lengths[strings[1]]
                        
                    if my_species not in species_names_taxid_length:
                        species_names_taxid_length[my_species] = {}
                        species_names_taxid_length[my_species][taxid] = contig_lengths[strings[1]]
                    else:
                        if taxid not in species_names_taxid_length[my_species]:
                            species_names_taxid_length[my_species][taxid] = contig_lengths[strings[1]]
                        else:
                            species_names_taxid_length[my_species][taxid] += contig_lengths[strings[1]]
                        

In [ ]:
for species in species_names_taxid_length:
    sorted_taxids = {k: v for k, v in sorted(species_names_taxid_length[species].items(), reverse=True, key=lambda item: item[1])}
    species_names_taxid_length[species] = sorted_taxids
# species_names_taxid_length

# Download genomes of taxids

In [ ]:
taxid_dates = {}
taxid_urls = {}
taxid_file_path = {}
taxid_assembly_level = {}
taxid_present = {}

for taxid in taxid_list:
    taxid_urls[taxid] = ""
    taxid_file_path[taxid] = ""
    taxid_present[taxid] = False

with open("/media/vijinim/data/Experiments/Data/Bacteria_Refs/assembly_summary.txt") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    line_count = 0
    
    for row in csv_reader:
        
        if not row[0].startswith("#"):
            
            if row[5] in taxid_list:
                
                name = row[0]
                taxid = row[5]
                version_status = row[10]
                assembly_level = row[11]
                genome_rep = row[13]
                rel_date = row[14].split("/")
                url = row[19]
                
                myurl = url+"/"+url.split("/")[-1]+"_genomic.fna.gz"
                            
                local_file = url.split("/")[-1]+"_genomic.fna.gz"

                myfile_name = project_path+"Assemblies/"+url.split("/")[-1]+"_genomic.fna"
                
                if version_status == "latest" and genome_rep == "Full":
                
                    if taxid not in taxid_dates:

                        if assembly_level in ["Complete Genome", "Contig", "Chromosome"]:

                            if os.path.exists("/media/vijinim/data/Experiments/Data/Bacteria_Refs/bacterial_references/"+local_file):

                                subprocess.run("cp /media/vijinim/data/Experiments/Data/Bacteria_Refs/bacterial_references/"+local_file+" "+project_path+"Assemblies", shell=True)

                            else:
                                subprocess.run("rsync --copy-links --times --verbose "+myurl.replace("ftp:", "rsync:")+" "+project_path+"Assemblies", shell=True)
#                                 subprocess.run("wget "+myurl, shell=True)
#                                 subprocess.run("mv "+local_file+" "+project_path+"Assemblies", shell=True)

                            if os.path.exists(project_path+"Assemblies/"+local_file):

                                try:

                                    with gzip.open(project_path+"Assemblies/"+local_file, 'rb') as f_in:
                                        with open(myfile_name, 'wb') as f_out:
                                            shutil.copyfileobj(f_in, f_out)

                                    subprocess.run("rm "+project_path+"Assemblies/"+local_file, shell=True)

                                    taxid_dates[taxid] = rel_date
                                    taxid_urls[taxid] = url
                                    taxid_file_path[taxid] = myfile_name
                                    taxid_assembly_level[taxid] = assembly_level
                                    taxid_present[taxid] = True

                                except:

                                    if os.path.exists(project_path+"Assemblies/"+local_file):
                                        subprocess.run("rm "+project_path+"Assemblies/"+local_file, shell=True)


                    else:
                        present = datetime.now()
                        

                        old_diff = present - datetime(int(taxid_dates[taxid][0]), int(taxid_dates[taxid][1]), int(taxid_dates[taxid][2]))
                        new_diff = present - datetime(int(rel_date[0]), int(rel_date[1]), int(rel_date[2]))

                        if old_diff > new_diff:

                            if not ((assembly_level!="Complete Genome" and taxid_assembly_level[taxid]=="Complete Genome") or (assembly_level=="Contig" and taxid_assembly_level[taxid]=="Chromosome")):


                                if not os.path.exists(project_path+"Assemblies/"+local_file):

                                    if os.path.exists("/media/vijinim/data/Experiments/Data/Bacteria_Refs/bacterial_references/"+local_file):

                                        subprocess.run("cp /media/vijinim/data/Experiments/Data/Bacteria_Refs/bacterial_references/"+local_file+" "+project_path+"Assemblies", shell=True)

                                    else:
                                        subprocess.run("rsync --copy-links --times --verbose "+myurl.replace("ftp:", "rsync:")+" "+project_path+"Assemblies", shell=True)
#                                         subprocess.run("wget "+myurl, shell=True)
#                                         subprocess.run("mv "+local_file+" "+project_path+"Assemblies", shell=True)

                                    if os.path.exists(project_path+"Assemblies/"+local_file):

                                        try:

                                            with gzip.open(project_path+"Assemblies/"+local_file, 'rb') as f_in:
                                                with open(myfile_name, 'wb') as f_out:
                                                    shutil.copyfileobj(f_in, f_out)

                                            if taxid_file_path[taxid] != "":
                                                if os.path.exists(taxid_file_path[taxid]):
                                                    subprocess.run("rm "+taxid_file_path[taxid], shell=True)

                                            subprocess.run("rm "+project_path+"Assemblies/"+local_file, shell=True)

                                            taxid_dates[taxid] = rel_date
                                            taxid_urls[taxid] = url
                                            taxid_file_path[taxid] = myfile_name
                                            taxid_assembly_level[taxid] = assembly_level
                                            taxid_present[taxid] = True

                                        except:

                                            if os.path.exists(project_path+"Assemblies/"+local_file):
                                                subprocess.run("rm "+project_path+"Assemblies/"+local_file, shell=True)
       

# Download genomes of missed species

In [ ]:
species_path = {}
not_present_species = []

present_count = 0

for ref in species_names_taxid_length:
    file_present = False
    for taxid in species_names_taxid_length[ref].keys():
        if taxid_file_path[taxid] != "":
            file_present = True
            species_path[ref] = taxid_file_path[taxid]
            break
    if file_present:
        present_count += 1
    else:
        print(ref, "not present", list(species_names_taxid_length[ref].keys())[0])
        not_present_species.append(ref)
        
print(present_count, len(not_present_species))

# Rename and copy species

In [ ]:
for taxid in taxid_present:
    if taxid_present[taxid]:
        subprocess.run("cp "+taxid_file_path[taxid]+" "+project_path+"Reference_Sequences/"+taxid+".fna", shell=True)
        
        

# Get length of ref

In [ ]:
taxid_file_len = {}

for taxid in taxid_present:
    command = 'grep -v ">" '+taxid_file_path[taxid]+' | wc | awk \'{print $3-$1}\''
    n = subprocess.check_output(command, shell=True)

    taxid_file_len[taxid] = int(n.decode("utf-8").strip())
        

# Get coverage

In [ ]:
contig_coverages = {}

with open(project_path+"contig_coverages.tsv", "r") as myfile:
    
    for line in myfile.readlines():
        
        if not line.startswith("Contig"):
            
            strings = line.strip().split()
        
            contig_coverages[strings[0]] = float(strings[1])

# Get species length and total contig length

In [ ]:
species_ref_length = {}
species_contig_length = {}

for species in species_names_taxid_length:
    species_ref_length[species] = 0
    species_contig_length[species] = 0

for species in species_names_taxid_length:
        
    total_sum = 0
    total_contig_length_sum = 0
    species_count = 0

    for taxid in species_names_taxid_length[species]:
        
        if taxid in taxid_file_len:
            if taxid_file_len[taxid] != 0:
                total_sum += taxid_file_len[taxid]
                total_contig_length_sum += species_names_taxid_length[species][taxid]
                species_count += 1
    
    species_contig_length[species] = total_contig_length_sum
    
    if species_count != 0:
        species_ref_length[species] = total_sum/species_count
    else:
        species_ref_length[species] = 0
        

# Get species coverage

In [ ]:
species_coverages = {}

for species in species_names_taxid_length:
    species_coverages[species] = 0

for species in species_names_taxid_length:
        
    total_sum = 0
    total_contig_length = 0

    for taxid in species_names_taxid_length[species]:

        for contig in taxid_contigs[taxid]:
            total_sum += contig_lengths[contig]*contig_coverages[contig]
            total_contig_length += contig_lengths[contig]

    if total_contig_length != 0:
        species_coverages[species] = total_sum/total_contig_length
    else:
        species_coverages[species] = 0


# Get species reference genome coverage

In [ ]:
species_genome_coverages = {}

for species in species_names_taxid_length:
    species_genome_coverages[species] = 0

for species in species_names_taxid_length:
    
    total_contig_length = 0

    for taxid in species_names_taxid_length[species]:
        
        for contig in taxid_contigs[taxid]:
            total_contig_length += contig_lengths[contig]
            
    if total_contig_length != 0 and species_ref_length[species] != 0:
        species_genome_coverages[species] = total_contig_length/species_ref_length[species]
    else:
        species_genome_coverages[species] = 0


# Get species relative abundance

In [ ]:
species_rel_abundance = {}

for species in species_names_taxid_length:
    species_rel_abundance[species] = 0
    
total_reads_mapped = 0

for species in species_names_taxid_length:
        
    species_reads = 0

    for taxid in species_names_taxid_length[species]:
        
        for contig in taxid_contigs[taxid]:
            species_reads += contig_lengths[contig]*contig_coverages[contig]
            total_reads_mapped += contig_lengths[contig]*contig_coverages[contig]
    
    species_rel_abundance[species] = species_reads
    
for species in species_names_taxid_length:
    species_rel_abundance[species] = species_rel_abundance[species]/total_reads_mapped

sorted_species = {k: v for k, v in sorted(species_rel_abundance.items(), reverse=True, key=lambda item: item[1])}
species_rel_abundance = sorted_species

# Get species taxanomic abundance

In [ ]:
species_tax_abundance = {}

for species in species_names_taxid_length:
    species_tax_abundance[species] = 0
    
total_coverage = 0

for species in species_names_taxid_length:
    
    species_tax_abundance[species] = species_coverages[species]
    total_coverage += species_coverages[species]
    
for species in species_names_taxid_length:
    species_tax_abundance[species] = species_tax_abundance[species]/total_coverage

sorted_species = {k: v for k, v in sorted(species_tax_abundance.items(), reverse=True, key=lambda item: item[1])}
species_tax_abundance = sorted_species


# Select and copy species above threshold

In [ ]:
s_n = 1

total_tax_abundance = 0

folder="Species_12/"

print("Species \t Relative abundance \t Taxonomic abundance \t Genome coverage")

with open(project_path+folder+"species_stats.tsv", "w") as myfile:
    
    myfile.write("Species name\tRelative abundance\tTaxonomic abundance\tGenome coverage\tSpecies coverage\n")
    
    for species in species_rel_abundance:

        if species_rel_abundance[species] > 0.005:
            
            myfile.write(species+"\t"+str(species_rel_abundance[species])+"\t"+str(species_tax_abundance[species])+"\t"+str(species_genome_coverages[species])+"\t"+str(species_coverages[species])+"\n")

            total_tax_abundance += species_tax_abundance[species]
            
            for taxid in species_names_taxid_length[species]:

                subprocess.run("cp "+taxid_file_path[taxid]+" "+project_path+folder+"Reference_Sequences/"+taxid+".fna", shell=True)

            print(s_n, species, "\t", species_rel_abundance[species], "\t", species_tax_abundance[species], "\t", species_genome_coverages[species], "\t", species_coverages[species])
            s_n+=1
        
# print("total_tax_abundance:", total_tax_abundance)

# Build MD3 database

## Get gene sequences

In [ ]:
n_species = folder

ppath = project_path
fpath = project_path+n_species+"final.contigs.fa.lst"


def get_seqs(path):
    active = ""
    
    for line in open(path):
        if line[0] == ">":
            active += line
        elif len(active) > 0 and len(line.strip()) != 0:
            active += line
        elif len(line.strip()) == 0 and len(active) > 0:
            yield active
            active = ""
    if len(active) > 0:
        yield active

gene_nt_seq = {}

for seq in get_seqs(fpath):
    if "_nt|" in seq:
        
        strings = seq.split("|")
        
        gene_id = strings[0][1:]
        
        gene_nt_seq[gene_id] = seq
        
with open(ppath+n_species+"/all_genes.fna", "w") as ntfile:
    for gene in gene_nt_seq:
        ntfile.write(gene_nt_seq[gene])
        


In [ ]:
gene_lengths = {}

lenthgs_genes = []

for index, record in enumerate(SeqIO.parse(project_path+n_species+"/all_genes.fna", "fasta")):
    
    strings = record.id.split("|")
        
    gene_id = strings[0]
    
#     print(gene_id)
    
    gene_lengths[gene_id] = len(record.seq)
    lenthgs_genes.append(len(record.seq))
    
print("Maximum gene length:", max(lenthgs_genes))
print("Minimum gene length:", min(lenthgs_genes))

# Align genes to refs

In [ ]:
!cat "$project_path""$folder"Reference_Sequences/*.fna > "$project_path""$folder"Reference_Sequences/refs.fna

!minimap2 -t 8 "$project_path""$folder"Reference_Sequences/refs.fna "$project_path""$folder"all_genes.fna > "$project_path""$folder"all_genes.paf

# !rm "$project_path""$folder"Reference_Sequences/refs.fna

contig_ref = defaultdict(list)
contig_ref_aln_length = defaultdict(list)
contig_length = {}

for line in open(project_path+folder+'all_genes.paf'):
    data = line.strip().split('\t')
#     1	string	Query sequence name
#     2	int	Query sequence length
#     3	int	Query start (0-based; BED-like; closed)
#     4	int	Query end (0-based; BED-like; open)
#     5	char	Relative strand: "+" or "-"
#     6	string	Target sequence name
#     7	int	Target sequence length
#     8	int	Target start on original strand (0-based)
#     9	int	Target end on original strand (0-based)
#     10	int	Number of residue matches
#     11	int	Alignment block length
#     12	int	Mapping quality (0-255; 255 for missing)

    qname = data[0]
    qlen = int(data[1])
    qstart = int(data[2])
    qqend = int(data[3])
    char = data[4]
    tname = data[5]
    tlen = int(data[6])
    tstart = int(data[7])
    aln_len = int(data[10])
    flag = int(data[11])

    if not flag == 255:

        contig_ref[qname].append(tname)
        contig_ref_aln_length[qname].append(aln_len)
        contig_length[qname] = qlen
        
        
threshold = 0.5

with open(project_path+folder+'all_genes.output', 'w+') as f:
    for k, v in contig_ref.items():
        best = None
        best_len = 0
        c_len = contig_length[k]

        if len(v) > 1:
            
            align_sum = 0
            
            for ref, l in zip(contig_ref[k], contig_ref_aln_length[k]):
                
                align_sum += l
                
            if align_sum >= threshold * c_len and best_len < align_sum:
                best_len = align_sum
                best = ref
                    
        elif contig_ref_aln_length[k][0] >= threshold * c_len:
            best = contig_ref[k][0]
            best_len = contig_ref_aln_length[k][0]
        else:
            best = 'POOR MAPPING'
        f.write(f'{k}\t{best}\t{best_len}\n')

# Get ref IDs

In [ ]:
reference_files = glob.glob(project_path+n_species+'/Reference_Sequences/*.fna')

ref_ids = {}

for ref in reference_files:
    ref_name = ref.split("/")[-1][:-4]
    pp = !grep "^>" "$ref"
    
    for item in pp:
        myid = item.split(" ")[0][1:]
        
        if ref_name not in ref_ids:
            ref_ids[ref_name] = [myid]
        else:
            ref_ids[ref_name].append(myid)
            
len(ref_ids)

# Get mapped species to genes

In [ ]:
mapped_gene_bins = {}

with open(project_path+n_species+'/all_genes.output', mode='r') as myfile:
    
    for line in myfile.readlines():
        
        strings = line.strip().split("\t")

        gene_num = strings[0].split("|")[0]
        
        for ref in ref_ids:
            if strings[1] in ref_ids[ref]:
                if gene_num not in mapped_gene_bins:
                    mapped_gene_bins[gene_num] = ref
                break
    
print(len(mapped_gene_bins))

# Get aa seqs

In [ ]:
genes_aa_list = {}
gene_seq = {}

genes_contigs = {}

gene_species_mapped = {}

unmapped_count = 0

mapped_gene_lengths = []
unmapped_gene_lengths = []

contigs_containing_genes = []

for seq in get_seqs(fpath):
    if "_aa|" in seq:
        
        strings = seq.split("|")
        
        gene_id = strings[0][1:]
        
        start_n = 'k141_'
        end_n = ''
        
        contig_id = seq.split(">")[-1].split(" ")[0]
        
        contig_num = int(re.search('%s(.*)%s' % (start_n, end_n), seq.split(">")[-1].split(" ")[0]).group(1))
        
        strings = seq.split(">")
        
        gene_id = strings[1].strip().split("|")[0]
        
        gene_seq[gene_id] = seq
        
        genes_contigs[gene_id] = contig_num
        
        has_bin = False

        if gene_id in mapped_gene_bins:
            
            if taxid_present[mapped_gene_bins[gene_id]]:

                gene_species_mapped[gene_id] = taxid_to_species[mapped_gene_bins[gene_id]]

                has_bin = True
                
            else:
                
                if contig_id in contig_taxid:
                    gene_species_mapped[gene_id] = taxid_to_species[contig_taxid[contig_id]]
                    has_bin = True

        if not has_bin:
            gene_species_mapped[gene_id] = "unmapped"
            unmapped_count += 1


In [ ]:
len(gene_species_mapped)

In [ ]:
unmapped_count

In [ ]:
print("Total number of genes:", len(gene_lengths))
print("Number of genes mapped:", len(gene_lengths)-unmapped_count)
print((len(gene_lengths)-unmapped_count)/len(gene_lengths))

In [ ]:
# Some data we want to write to the worksheet.
gene_species = {"GeneID":"Annotation"}

with open(project_path+n_species+"/all_genes.faa", "w") as aafile:
    for gene in gene_species_mapped:
        
        gene_id = gene
        
        if gene_species_mapped[gene] == "unmapped":
            gene_species[gene_id] = "-"
        else:
            gene_species[gene_id] = "["+gene_species_mapped[gene].replace("_", " ")+"]"
            
        aafile.write(gene_seq[gene])


In [ ]:
od = collections.OrderedDict(sorted(gene_species.items()))

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook(project_path+n_species+"/"+n_species.lower()[:-1]+".genes.species.mapped.xlsx")
worksheet = workbook.add_worksheet()

# Start from the first cell. Rows and columns are zero indexed.
row = 0
col = 0

print(len(od.items()))

# Iterate over the data and write it out row by row.
for gene, species in od.items():
    worksheet.write(row, col,     gene)
    worksheet.write(row, col + 1, species)
    row += 1
    
workbook.close()

# Get species count in contigs

In [ ]:
species_in_contig = {}

for gene in genes_contigs:
    
    if gene in gene_species_mapped:
        
        if gene_species_mapped[gene] != "unmapped":
                
            if genes_contigs[gene] not in species_in_contig:
                species_in_contig[genes_contigs[gene]] = [gene_species_mapped[gene]]
            else:
                if gene_species_mapped[gene] not in species_in_contig[genes_contigs[gene]]:
                    species_in_contig[genes_contigs[gene]].append(gene_species_mapped[gene])


In [ ]:
counts_species_in_contig = {}

for contig in species_in_contig:
    
    if len(species_in_contig[contig]) not in counts_species_in_contig:
        counts_species_in_contig[len(species_in_contig[contig])] = 1
    else:
        counts_species_in_contig[len(species_in_contig[contig])] += 1
        
counts_species_in_contig